In [9]:
import random 
import copy
import numpy as np
from client import *
import matplotlib.pyplot as plt


In [10]:
Team_Key = "VPN75t7UGO44XsrQlo9KuKkg8yRrqgSfyh13GITsQnyvLLyvV5" 
Team_Name = "Platypus_Perry"
Max_Len = 10 
Min_Len = -10
Generations = 10
Array_Length = 11
Population_Size = 8
Mutation_Probability = 0.2         # probabilty that it will mutate
Mutation_Difference_Scale = 1000  # adding noice on mutation to actual value ratio
Initial_Array = [0.0, -1.45799022e-12, -2.28980078e-13,  4.62010753e-11, -1.75214813e-10, -1.83669770e-15,  8.52944060e-16,  2.29423303e-05, -2.04721003e-06, -1.59792834e-08,  9.98214034e-10]


In [11]:
class Platypus:
    def __init__(self,arr):
        self.population = self.populate(arr)
        self.population[0] = np.array(arr)
        self.avg_fitness = [] 
        self.weight_fitness_map = []
        self.max_fitness_across_generations = None
        self.best_chromosome_across_generations = None
        self.train_for_best_chromosome = None
        self.valid_for_best_chromosome = None
        self.print_population()
    
    def populate(self,arr):
        temp = [list(arr) for i in range(Population_Size)]
        temp = np.array(temp, dtype=np.double)
        temp =  self.mutate(temp,0)
        temp[0] = arr
        return temp

    def print_population(self):
        for x in self.population:
            print(x,file=open("output.txt", "a"))

    def add_uniform_noise(self,arr):
        for idx, val in np.ndenumerate(arr):
            if np.random.random() < Mutation_Probability:
                range_lim = val/Mutation_Difference_Scale
                noise = np.random.uniform(-range_lim,range_lim)
                arr[idx] = arr[idx] + noise
        return np.clip(arr, Min_Len, Max_Len)
    
    def get_fitness(self):
        fitness = []
        train_errors = []
        valid_errors = []
        weight_fitness = []
        for chrom in self.population:
            train_err, valid_err = get_errors(Team_Key, list(chrom))
            fit = -(train_err + valid_err)
            fitness.append(fit)
            print(chrom)
            print(fit)
            train_errors.append(train_err)
            valid_errors.append(valid_err)
            weight_fitness.append((chrom,fit))
        fitness = np.array(fitness, dtype=np.double)
        self.weight_fitness_map = weight_fitness
        return fitness, train_errors, valid_errors

    def add_gauss_noise(self,arr):
        means = np.mean(arr, axis=0) 
        for idx, val in np.ndenumerate(arr):
            if np.random.random() < Mutation_Probability:
               noise = np.random.normal(loc=means[idx[1]], scale=abs(means[idx[1]]/1000))
               arr[idx] = arr +  noise
        return np.clip(arr, Min_Len, Max_Len)
    
    def mutate(self, arr, flag=0):
        temp_pop = copy.deepcopy(arr)
        if(flag < 0):
            temp = self.add_gauss_noise(arr)
        if(flag >= 0):
            temp = self.add_uniform_noise(arr)            
        return temp

    def gauss_index(thresholds):
        draw = np.random.random()
        for i in range(len(thresholds)):
            if draw < thresholds[i]:
                return i
        return len(thresholds) - 1 
    
    def next_gen(self):
        fitness, train_errors, valid_errors = self.get_fitness()
        normalized_fitness = (fitness - np.min(fitness)) / np.ptp(fitness) 
        self.avg_fitness.append(np.mean(fitness))
        self.update_best(fitness, train_errors, valid_errors)
        thresholds = []
        thresh = 0.0
        fitness_sum = np.sum(normalized_fitness)
        for val in normalized_fitness:
            thresh = thresh + (val/fitness_sum)
            thresholds.append(thresh)
        bette = []
        for i in range(int(self.POPULATION_SIZE/2)):
            motera = self.population[gauss_index(thresholds)]
            modiji = self.population[gauss_index(thresholds)]
            adani, reliance = self.crossover(motera, mogiji)
            bette.append(adani)
            bette.append(reliance)
         bette = np.array(bette, dtype=np.double)
         self.population = self.mutate(bette)


In [12]:
platypus = Platypus(Initial_Array)

In [ ]:
 for i in range(Generations):
    platypus.next_gen()
